In [1]:
keys = [
        "213d94fbcbe941269843d7d2b26440b0",
        "e39c640df8fc4b489eac62b6e6150d06",
        "ebd1991e97d947ff89f3516fcbcd8ad1",
        "2209d64f0a3f42a4b9403aad4ba2f5e9",
        "e236bd0eefd2424494000ab3fcc6bb77",
        "bf864af3c3314584b911d10f35b04ce3",
        "862bc36fc5ab48e0bbfe1a4bfe2cedf4",
        "d686bf1d0bd34f7abb0cf77d831d100a",
        "8b2da099801743e29bf049329c2bdff6",
        "f4118e379d4243c7bb7efad796785d2c"
        ]

In [2]:
import requests
import json
import pandas as pd
import numpy as np

In [26]:
def bls_pull(registration_key, area_code, start_year, end_year):
    
    



    industries = {'11': 'natural_resources_mining',
                  '12': 'construction',
                  '13': 'manufacturing',
                  '21': 'trade_transportation_utilities',
                  '22': 'information',
                  '23': 'financial_activities',
                  '24': 'professional_business_services',
                  '25': 'education_health_services',
                  '26': 'leisure_hospitality',
                  '27': 'other_services',
                  '28': 'public_administration',
                  '29': 'unclassified'}

    data_names = {'1': 'all_employees',
                  '4': 'avg_weekly_pay'}




    json_dicts = {}

    for i_code, i_name in industries.items():

        for d_code, d_name in data_names.items():

            headers = {'Content-type': 'application/json'}
            data = json.dumps({"seriesid": ['ENU'+ area_code + d_code + '0510' + i_code],'registrationkey': registration_key, 
                               "startyear": start_year, "endyear": end_year})
            p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
            json_data = json.loads(p.text)

            json_dicts[i_name + '_' + d_name] = json_data['Results']['series'][0]['data']

    bad_keys = []
    for i in json_dicts:
        if len(json_dicts[i]) == 0:
            bad_keys.append(i) 

    for i in bad_keys:
        del json_dicts[i]




    df_dicts = {}

    for i in json_dicts:
        df_dicts[i] = pd.DataFrame(json_dicts[i])

    for i in df_dicts:

        if 'avg_weekly_pay' in i:
            df_dicts[i][i] = df_dicts[i]['value']
            df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
            df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

        elif 'all_employees' in i:
            df_dicts[i][i] = df_dicts[i]['value']
            df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
            df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

        cols = [j for j in df_dicts[i] if j != 'date']
        df_dicts[i][cols] = df_dicts[i][cols].apply(pd.to_numeric, errors='coerce')

    for df_key in df_dicts:

        wp_mask = df_key.split('all')[0] + 'avg_weekly_pay'

        if 'all_employees' in df_key:

            df_dicts[df_key][wp_mask] = 0

            for index, period in df_dicts[df_key]['date'].iteritems():
                if period[4:] in ['01', '02', '03']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'01')][wp_mask].iloc[0]
                if period[4:] in ['04', '05', '06']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'02')][wp_mask].iloc[0]
                if period[4:] in ['07', '08', '09']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'03')][wp_mask].iloc[0]
                if period[4:] in ['10', '11', '12']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'04')][wp_mask].iloc[0]

            df_dicts[df_key].rename(columns={wp_mask: df_key.split('all')[0] + 'avg_monthly_pay'}, inplace=True)
            df_dicts[df_key][df_key.split('all')[0] + 'aggr_value'] = df_dicts[df_key][df_key] * df_dicts[df_key][df_key.split('all')[0] + 'avg_monthly_pay']

    merged_df = pd.DataFrame(columns=['date'])

    for df_key in df_dicts:
        if 'all_employees' in df_key:
            merged_df = pd.merge(merged_df, df_dicts[df_key], how = 'outer', on ='date')

    temp_merge = merged_df.drop('date', axis=1)
    temp_cols = [i for i in temp_merge.columns if 'aggr_val' in i]
    merged_df['total_industries_aggr_value'] = temp_merge[temp_cols].apply(lambda x: sum(x), axis=1)
    
    #Reverses Dates:
    merged_df = merged_df[::-1]
    #Formates Dates year and month split
    merged_df['date'] = [x[:4]+'-'+x[4:8] for x in merged_df['date']]
    
    return pd.DataFrame(merged_df)

In [6]:
# def bls_pull(registration_key, area_code, start_year, end_year):
    
#     industries = {
#         "1011" : "natural_resources_and_mining",
#         "1012" : "construction",
#         "1013" : "manufacturing",
#         "1021" : "trade_transportation_and_utilities",
#         "1022" : "information",
#         "1023" : "financial_activities",
#         "1024" : "professional_and_business_services",
#         "1025" : "education_and_health_services",
#         "1026" : "leisure_and_hospitality",
#         "1027" : "other_services",
#         "1028" : "public_administration",
#         "1029" : "unclassified"
#     }

# #     owners = {
# #         "0" : "total_covered",
# #         "5" : "private",
# #         "3" : "local_govt",
# #         "2" : "state_govt",
# #         "1" : "federal_govt",
# #         "8" : "total_govt",
# #         "9" : "total_ui_covered" # excludes federal govt
# #     }

#     data_names = {
#         "1" : "all_employees",
#         "4" : "avg_weekly_wage"
#     }



#     json_dicts = {}

#     for i_code, i_name in industries.items():

#         for d_code, d_name in data_names.items():

#             headers = {'Content-type': 'application/json'}
#             data = json.dumps({"seriesid": ['ENU'+ area_code + d_code + '0' + "0" + i_code],'registrationkey': registration_key, 
#                                "startyear": start_year, "endyear": end_year})
#             p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
#             json_data = json.loads(p.text)

#             json_dicts[i_name + '_' + d_name] = json_data['Results']['series'][0]['data']

#     bad_keys = []
#     for i in json_dicts:
#         if len(json_dicts[i]) == 0:
#             bad_keys.append(i) 

#     for i in bad_keys:
#         del json_dicts[i]




#     df_dicts = {}

#     for i in json_dicts:
#         df_dicts[i] = pd.DataFrame(json_dicts[i])

#     for i in df_dicts:

#         if 'avg_weekly_pay' in i:
#             df_dicts[i][i] = df_dicts[i]['value']
#             df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
#             df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

#         elif 'all_employees' in i:
#             df_dicts[i][i] = df_dicts[i]['value']
#             df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
#             df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

#         cols = [j for j in df_dicts[i] if j != 'date']
#         df_dicts[i][cols] = df_dicts[i][cols].apply(pd.to_numeric, errors='coerce')

#     for df_key in df_dicts:

#         wp_mask = df_key.split('all')[0] + 'avg_weekly_pay'

#         if 'all_employees' in df_key:

#             df_dicts[df_key][wp_mask] = 0

#             for index, period in df_dicts[df_key]['date'].iteritems():
#                 if period[4:] in ['01', '02', '03']:
#                     df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'01')][wp_mask].iloc[0]
#                 if period[4:] in ['04', '05', '06']:
#                     df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'02')][wp_mask].iloc[0]
#                 if period[4:] in ['07', '08', '09']:
#                     df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'03')][wp_mask].iloc[0]
#                 if period[4:] in ['10', '11', '12']:
#                     df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'04')][wp_mask].iloc[0]

#             df_dicts[df_key].rename(columns={wp_mask: df_key.split('all')[0] + 'avg_monthly_pay'}, inplace=True)
#             df_dicts[df_key][df_key.split('all')[0] + 'aggr_value'] = df_dicts[df_key][df_key] * df_dicts[df_key][df_key.split('all')[0] + 'avg_monthly_pay']

#     merged_df = pd.DataFrame(columns=['date'])

#     for df_key in df_dicts:
#         if 'all_employees' in df_key:
#             merged_df = pd.merge(merged_df, df_dicts[df_key], how = 'outer', on ='date')

#     temp_merge = merged_df.drop('date', axis=1)
#     temp_cols = [i for i in temp_merge.columns if 'aggr_val' in i]
#     merged_df['total_industries_aggr_value'] = temp_merge[temp_cols].apply(lambda x: sum(x), axis=1)
    
#     #Reverses Dates:
#     merged_df = merged_df[::-1]
#     #Formates Dates year and month split
#     merged_df['date'] = [x[:4]+'-'+x[4:8] for x in merged_df['date']]
    
#     return pd.DataFrame(merged_df)http://localhost:8888/notebooks/BLS-API.ipynb#Atlantic-City

### Atlantic City, NJ

In [27]:
city = "AtlanticCity"
start = "2010"
end = "2015"
code= "C1210"

In [30]:
acnj = bls_pull(registration_key=keys[2], start_year=start, end_year=end, area_code=code)

In [31]:
acnj.head().T

,71,70,69,68,67
date,2010-01,2010-02,2010-03,2010-04,2010-05
natural_resources_mining_all_employees,402,346,455,607,731
natural_resources_mining_avg_monthly_pay,2324,2324,2324,1480,1480
natural_resources_mining_aggr_value,934248,804104,1057420,898360,1081880
construction_all_employees,4124,3829,4133,4369,4591
construction_avg_monthly_pay,4172,4172,4172,4320,4320
construction_aggr_value,17205328,15974588,17242876,18874080,19833120
manufacturing_all_employees,2066,2023,2095,2160,2227
manufacturing_avg_monthly_pay,3568,3568,3568,3756,3756
manufacturing_aggr_value,7371488,7218064,7474960,8112960,8364612


In [32]:
acnj.to_csv("./bls-api-data/" +str(city)+str(start)+'-'+str(end)+".csv")

### Bridgeport, CT

In [33]:
city = "Bridgeport"
start = "2010"
end = "2015"
code= "C1486"

In [34]:
bpct = bls_pull(registration_key=keys[2], start_year=start, end_year=end, area_code=code)

In [35]:
bpct.head().T

,71,70,69,68,67
date,2010-01,2010-02,2010-03,2010-04,2010-05
natural_resources_mining_all_employees,329,326,342,359,375
natural_resources_mining_avg_monthly_pay,3376,3376,3376,3432,3432
natural_resources_mining_aggr_value,1110704,1100576,1154592,1232088,1287000
construction_all_employees,9513,9322,9523,10830,11166
construction_avg_monthly_pay,4244,4244,4244,4232,4232
construction_aggr_value,40373172,39562568,40415612,45832560,47254512
manufacturing_all_employees,37362,37263,37309,37280,37468
manufacturing_avg_monthly_pay,7416,7416,7416,6216,6216
manufacturing_aggr_value,277076592,276342408,276683544,231732480,232901088


In [36]:
bpct.to_csv("./bls-api-data/" +str(city)+str(start)+'-'+str(end)+".csv")

### New Haven, CT

In [37]:
city = "NewHaven"
start = "2010"
end = "2015"
code= "C3530"

In [38]:
nhct = bls_pull(registration_key=keys[2], start_year=start, end_year=end, area_code=code)

In [39]:
nhct.head().T

,71,70,69,68,67
date,2010-01,2010-02,2010-03,2010-04,2010-05
natural_resources_mining_all_employees,692,669,786,1060,1137
natural_resources_mining_avg_monthly_pay,2548,2548,2548,2724,2724
natural_resources_mining_aggr_value,1763216,1704612,2002728,2887440,3097188
construction_all_employees,10373,10122,10447,11465,11891
construction_avg_monthly_pay,4352,4352,4352,4356,4356
construction_aggr_value,45143296,44050944,45465344,49941540,51797196
manufacturing_all_employees,33348,33227,33377,33414,33587
manufacturing_avg_monthly_pay,4820,4820,4820,4564,4564
manufacturing_aggr_value,160737360,160154140,160877140,152501496,153291068


In [40]:
nhct.to_csv("./bls-api-data/" +str(city)+str(start)+'-'+str(end)+".csv")

### New London, CT

In [41]:
city = "NewLondon"
start = "2010"
end = "2015"
code= "C3598"

In [42]:
nlct = bls_pull(registration_key=keys[2], start_year=start, end_year=end, area_code=code)

In [43]:
nlct.head().T

,71,70,69,68,67
date,2010-01,2010-02,2010-03,2010-04,2010-05
natural_resources_mining_all_employees,869,868,897,1230,1255
natural_resources_mining_avg_monthly_pay,2312,2312,2312,2392,2392
natural_resources_mining_aggr_value,2009128,2006816,2073864,2942160,3001960
construction_all_employees,2734,2880,2910,3450,3255
construction_avg_monthly_pay,4012,4012,4012,4156,4156
construction_aggr_value,10968808,11554560,11674920,14338200,13527780
manufacturing_all_employees,14430,14288,14280,14209,14293
manufacturing_avg_monthly_pay,6632,6632,6632,6072,6072
manufacturing_aggr_value,95699760,94758016,94704960,86277048,86787096


In [44]:
nlct.to_csv("./bls-api-data/" +str(city)+str(start)+'-'+str(end)+".csv")

### Boston, MA

In [45]:
city = "Boston"
start = "2010"
end = "2015"
code= "C1446"

In [46]:
boma = bls_pull(registration_key=keys[2], start_year=start, end_year=end, area_code=code)

In [47]:
boma.head().T

,71,70,69,68,67
date,2010-01,2010-02,2010-03,2010-04,2010-05
natural_resources_mining_all_employees,3208,3146,3310,4014,4326
natural_resources_mining_avg_monthly_pay,3900,3900,3900,3160,3160
natural_resources_mining_aggr_value,1.25112e+07,1.22694e+07,1.2909e+07,1.26842e+07,1.36702e+07
construction_all_employees,68684,66513,67780,73080,76475
construction_avg_monthly_pay,4776,4776,4776,4776,4776
construction_aggr_value,3.28035e+08,3.17666e+08,3.23717e+08,3.4903e+08,3.65245e+08
manufacturing_all_employees,177327,176680,176809,177217,178016
manufacturing_avg_monthly_pay,6100,6100,6100,6144,6144
manufacturing_aggr_value,1.08169e+09,1.07775e+09,1.07853e+09,1.08882e+09,1.09373e+09


In [48]:
boma.to_csv("./bls-api-data/" +str(city)+str(start)+'-'+str(end)+".csv")

### Newport, RI

In [49]:
city = "Newport"
start = "2010"
end = "2015"
code= "44005"

In [50]:
npri = bls_pull(registration_key=keys[2], start_year=start, end_year=end, area_code=code)

In [51]:
npri.head().T

,71,70,69,68,67
date,2010-01,2010-02,2010-03,2010-04,2010-05
natural_resources_mining_all_employees,NaN,NaN,NaN,233,263
natural_resources_mining_avg_monthly_pay,NaN,NaN,NaN,1964,1964
natural_resources_mining_aggr_value,NaN,NaN,NaN,457612,516532
construction_all_employees,NaN,NaN,NaN,1444,1535
construction_avg_monthly_pay,NaN,NaN,NaN,3188,3188
construction_aggr_value,NaN,NaN,NaN,4.60347e+06,4.89358e+06
manufacturing_all_employees,2362,2363,2368,2389,2404
manufacturing_avg_monthly_pay,6144,6144,6144,5256,5256
manufacturing_aggr_value,14512128,14518272,14548992,12556584,12635424


In [52]:
npri.to_csv("./bls-api-data/" +str(city)+str(start)+'-'+str(end)+".csv")